# MNIST 手寫數字辨識 (MNIST_CNN_TF)

2017/06/26  
徐仕杰 

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
- 好用的markdown語法
[markdown](https://www.zybuluo.com/codeep/note/163962#1如何输入一个方程式序列)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas
- import lzma error: 請用python 3


## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. 開始Deep Learning - CNN](#開始Deep Learning)  
  -  [A. 定義參數](#定義參數) 
  -  [B. 設計一個Model從X預測Y](#設計一個) 
  -  [C. 選一個loss function,](#選一個loss) 
  -  [D. 選一個optimizer](#選一個o) 
  -  [E. 開始執行訓練](#開始執行) 
  -  [F. 算一下正確率](#算一下正)  
<br>
-  [3. Reference](#reference)

<a id='import'></a>
## Import Package & Functions

In [1]:
import os
import sys
import numpy as np
from PIL import Image
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tensorflow.python.client import device_lib
import pandas as pd

In [2]:
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data
#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [5]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')
    
print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])

list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [6]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [7]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='開始Deep Learning'></a>
## 2. 開始Deep Learning - CNN
- What is CNN?  
![](img/cnn_1.gif)
![](img/cnn_2.png)

- Deep Learning ABC 
    -  [A. 定義參數](#定義參數) 
    -  [B. 設計一個Model從X預測Y](#設計一個) 
    -  [C. 選一個loss function,](#選一個loss) 
    -  [D. 選一個optimizer](#選一個o) 
    -  [E. 開始執行訓練](#開始執行) 
    -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [8]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, 784], name="X")
Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Y_")

In [9]:
# 設定 weight 和 bais
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    W = tf.Variable(initial, name ='W')
    variable_summaries(W,'W')
    return W
def bias_variable(shape):
    initial = tf.constant(value=0.1, shape=shape)
    B = tf.Variable(initial, name = 'b')
    variable_summaries(B,'B')
    return B
# 設定 cnn 的 layers
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME',use_cudnn_on_gpu=True)
    #stride:每次移動步伐, padding:在圖片的四周補0, SAME就是補到conv後要和原圖一樣大張
def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    #ksize:框框大小, stride:每次移動步伐

<a id='設計一個'></a>
###  B. 設計一個Model從X & X history預測Y  
Input Layer: X  
Hidden Layer 1: $H_1=pooling(f(W_1(X)+B_1)$  
Hidden Layer 2: $H_2=pooling(f_2(W_2H_1+B_2))$  
Output Layer : $Y=pooling(f_3(W_3H_2+B_3))$  


### L1. Input Layer 輸入層
- do nothing

### L2. Hidden Layer 隱藏層 x 3

In [10]:
# fisrt layer
with tf.name_scope('conv1'):
    ## variables
    W_conv1 = weight_variable([3,3,1,32]) # filter_height, filter_width, in_channels, out_channels
    b_conv1 = bias_variable([32])
    ## build the layer
    X_image = tf.reshape(X, [-1, 28, 28,1]) # batch, in_height, in_width, in_channels
    h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)


In [11]:
# second layer
with tf.name_scope('conv2'):
    ## variables
    W_conv2 = weight_variable([3,3,32,64])
    b_conv2 = bias_variable([64])
    ## build the layer
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

In [12]:
# fully-connected layer
with tf.name_scope('full'):
    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

### L3. Output Layer 輸出層

In [13]:
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout
with tf.name_scope('output'):
    W_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])
    _pred = tf.matmul(h_fc1_drop, W_fc2)+b_fc2
    pred = tf.nn.softmax(_pred, name="pred")

with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1), name="correction")
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name="accuracy")

<a id='選一個loss'></a>
###  C. 選一個loss function, 當作Machine learning的目標
- cross_entorpy $-log(\Pr(Y_{true}))$

In [14]:
loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=_pred))

<a id='選一個o'></a>
### D. 選一個optimizer, 根據Data和我們訂的目標找參數W, B

In [15]:
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

<a id='開始執行'></a>
### E. 開始執行訓練(Training Data + Validataion Data)

In [16]:
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
with tf.name_scope('performance'):  
    loss_scar = tf.summary.scalar('loss', loss)
    # Create a summary to monitor accuracy tensor
    acc_scar = tf.summary.scalar('accuracy', accuracy)
    # Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [ ]:
sess = tf.InteractiveSession()
sess.run(init)

if not os.path.exists('tb-log_6'):
    os.mkdir('tb-log_6')
summary_writer_train = tf.summary.FileWriter('tb-log1/train',graph=tf.get_default_graph())
summary_writer_validation = tf.summary.FileWriter('tb-log1/validation',graph=tf.get_default_graph())

In [ ]:
#%%timeit -r 1 -n 1
epoch = 15
batch_size = 128
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        _, loss_s, acc_s, summary= sess.run([optimizer, loss_scar, acc_scar, merged_summary_op], feed_dict={X: batch_x, Y_:batch_y, keep_prob:0.5})
        summary_writer_train.add_summary(summary, ep * total_batch + i)
        if i%100 ==0:
            loss_v, acc_v, loss_s, acc_s, summary= sess.run([loss, accuracy, loss_scar, acc_scar, merged_summary_op], feed_dict={X: validation_X , Y_: validation_Y, keep_prob:1.0 })
            #acc = accuracy.eval({X: validation_X , Y_: validation_Y, keep_prob:1.0})
            #los=loss.eval({X: validation_X , Y_: validation_Y, keep_prob:1.0})
            summary_writer_validation.add_summary(summary, ep * total_batch + i)
            updateProgress('epoch:{x0}, batch:{x4} loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc_v,x3=loss_v,x4=i))
    print()


epoch:0, batch:300 loss:0.2522891163825989, acc:0.93190008401870731
epoch:1, batch:300 loss:0.15260650217533112, acc:0.9574002027511597
epoch:2, batch:300 loss:0.11232933402061462, acc:0.9688001275062561
epoch:3, batch:300 loss:0.08874470740556717, acc:0.9756001234054565
epoch:4, batch:300 loss:0.0763041079044342, acc:0.97780013084411625
epoch:5, batch:300 loss:0.06734831631183624, acc:0.9805001020431519
epoch:6, batch:300 loss:0.06283014267683029, acc:0.9819001555442813
epoch:7, batch:300 loss:0.058500535786151886, acc:0.9830001592636108
epoch:8, batch:300 loss:0.0523262657225132, acc:0.984800159931182954
epoch:9, batch:300 loss:0.04945176839828491, acc:0.98590016365051272
epoch:10, batch:300 loss:0.046321868896484375, acc:0.9863001108169556
epoch:11, batch:300 loss:0.045690495520830154, acc:0.9862001538276672
epoch:12, batch:300 loss:0.041703756898641586, acc:0.987800121307373
epoch:13, batch:300 loss:0.04176029562950134, acc:0.98740011453628543
epoch:14, batch:300 loss:0.04096240922

In [ ]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = pred.eval(feed_dict={X: train_X[21:30], keep_prob:1.0})
pred_tmp_df = pd.DataFrame(pred_tmp)
pred_tmp_df

In [ ]:
_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(train_X[21:30])
print('answer: ',train_y[21:30])

print('第21~30筆資料的比對結果:',pd.DataFrame(_))

_ = correct_pred.eval({X: train_X[21:30] , Y_: train_Y[21:30], keep_prob:1.0})
print('第21~30筆資料的比對結果:',pd.DataFrame(_))

<a id='算一下正'></a>
### F. 算一下正確率Accuracy (用Testing Data)
- unseen data
- generalize from the training set to the test set
- 泛化

In [ ]:
# predict all X and get the accuracy
_ = accuracy.eval({X: test_X , Y_: test_Y, keep_prob:1.0})
print('Accuracy:',_)

In [ ]:
#sess.close()